In [43]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [63]:
import sys
sys.path.append('DeepLatentMicrobiome/Src/')
from layers import *
from data import *
from train import *

In [72]:
auto, encoder_bioma, encoder_domain, decoder_bioma = autoencoder(domain_shape = None)

In [77]:
for layer in auto.layers:
    # Print the layer name
    print("Layer name:", layer.name)
    # Print the input shape of the layer
    print("Input shape:", layer.input_shape)
    # Print the output shape of the layer
    print("Output shape:", layer.output_shape)
    # Print the activation function of the layer
    try: 
        print("Activation function:", layer.activation.__name__)
    except:
        print("not activation")
    # Print the number of weights and biases of the layer
    try: 
        print("Number of weights:", len(layer.get_weights()[0]))
    except:
        print("no weights")
    try: 
        print("Number of biases:", len(layer.get_weights()[1]))
    except:
        print("no weights")
    # Print a blank line for readability
    print()

Layer name: bioma_input_717
Input shape: [(None, 717)]
Output shape: [(None, 717)]
not activation
no weights
no weights

Layer name: center_log_ratio
Input shape: (None, 717)
Output shape: (None, 717)
not activation
no weights
no weights

Layer name: encoder_bioma_dense_128
Input shape: (None, 717)
Output shape: (None, 128)
not activation
Number of weights: 717
Number of biases: 128

Layer name: encoder_bioma_dense_64
Input shape: (None, 128)
Output shape: (None, 64)
not activation
Number of weights: 128
Number of biases: 64

Layer name: encoded_bioma_10
Input shape: (None, 64)
Output shape: (None, 10)
Activation function: tanh
Number of weights: 64
Number of biases: 10

Layer name: decoder_dense_64
Input shape: (None, 10)
Output shape: (None, 64)
not activation
Number of weights: 10
Number of biases: 64

Layer name: decoder_dense_128
Input shape: (None, 64)
Output shape: (None, 128)
not activation
Number of weights: 64
Number of biases: 128

Layer name: decoded_bioma
Input shape: (Non

In [65]:
encoder_bioma.summary()

Model: "EncoderBioma"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bioma_input_717 (InputLayer  [(None, 717)]            0         
 )                                                               
                                                                 
 center_log_ratio (CenterLog  (None, 717)              0         
 Ratio)                                                          
                                                                 
 encoder_bioma_dense_128 (De  (None, 128)              91904     
 nse)                                                            
                                                                 
 encoder_bioma_dense_64 (Den  (None, 64)               8256      
 se)                                                             
                                                                 
 encoded_bioma_10 (Dense)    (None, 10)               

In [67]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 center_log_ratio (CenterLog  (None, 717)              0         
 Ratio)                                                          
                                                                 
 encode_128 (Dense)          (None, 128)               91904     
                                                                 
 encode_64 (Dense)           (None, 64)                8256      
                                                                 
 encode_10 (Dense)           (None, 10)                650       
                                                                 
Total params: 100,810
Trainable params: 100,810
Non-trainable params: 0
_________________________________________________________________


In [66]:
decoder_bioma.summary()

Model: "DecoderBioma"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 latent_space_input (InputLa  [(None, 10)]             0         
 yer)                                                            
                                                                 
 decoder_dense_64 (Dense)    (None, 64)                704       
                                                                 
 decoder_dense_128 (Dense)   (None, 128)               8320      
                                                                 
 decoded_bioma (Dense)       (None, 717)               92493     
                                                                 
Total params: 101,517
Trainable params: 101,517
Non-trainable params: 0
_________________________________________________________________


In [68]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decode_64 (Dense)           (None, 64)                704       
                                                                 
 decode_128 (Dense)          (None, 128)               8320      
                                                                 
 decode_717 (Dense)          (None, 717)               92493     
                                                                 
Total params: 101,517
Trainable params: 101,517
Non-trainable params: 0
_________________________________________________________________


In [89]:
otu_size = 717
latent_size = 10
hidden_layers = [128, 64]
encoder = keras.Sequential([
    CenterLogRatio(),
    layers.Dense(128,  name = "encode_128"),
    layers.Dense(64, name = "encode_64"),
    layers.Dense(10, activation = 'tanh', name = "encode_10")
], name = "encoder")
decoder = keras.Sequential([
    layers.Dense(64, input_shape = (10,),name = "decode_64"),
    layers.Dense(128,  name = "decode_128"),
    layers.Dense(717, activation = 'linear', name = "decode_717")
], name = "decoder")
input_layer = layers.Input(shape = (otu_size,), name = "input_layer")
latent_vector = encoder(input_layer)
output = decoder(latent_vector)


In [90]:
model = keras.Model(inputs = input_layer, outputs = output)
model.compile("adam", loss = "mean_squared_error")
model.summary(expand_nested = True)

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 717)]             0         
                                                                 
 encoder (Sequential)        (None, 10)                100810    
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| center_log_ratio (CenterLog  (None, 717)            0         |
| Ratio)                                                        |
|                                                               |
| encode_128 (Dense)        (None, 128)               91904     |
|                                                               |
| encode_64 (Dense)         (None, 64)                8256      |
|                                                               |
| encode_10 (Dense)         (None, 10)                650       |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯

In [91]:
X_train, X_test, __, __, otu_columns, __ = read_data(otu_filename = 'DeepLatentMicrobiome/Datasets/otu_table_all_80.csv', 
metadata_filename = 'DeepLatentMicrobiome/Datasets/metadata_table_all_80.csv')

In [92]:
model.fit(X_train, X_train, epochs = 50, batch_size=64)

Epoch 1/50
 1/67 [..............................] - ETA: 23s - loss: 384476.9375

2023-08-24 09:16:00.344062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 1s 9ms/step - loss: 416378.0938
Epoch 2/50
67/67 [==============================] - 1s 9ms/step - loss: 405233.3438
Epoch 3/50
67/67 [==============================] - 1s 7ms/step - loss: 371411.1875
Epoch 4/50
67/67 [==============================] - 1s 8ms/step - loss: 339515.1562
Epoch 5/50
67/67 [==============================] - 0s 7ms/step - loss: 323606.4688
Epoch 6/50
67/67 [==============================] - 1s 7ms/step - loss: 326151.1875
Epoch 7/50
67/67 [==============================] - 0s 7ms/step - loss: 326930.2188
Epoch 8/50
67/67 [==============================] - 0s 7ms/step - loss: 323745.8750
Epoch 9/50
67/67 [==============================] - 1s 9ms/step - loss: 304198.0938
Epoch 10/50
67/67 [==============================] - 1s 8ms/step - loss: 302771.9062
Epoch 11/50
67/67 [==============================] - 1s 8ms/step - loss: 307357.2500
Epoch 12/50
67/67 [==============================] - 1s 8ms/step - loss: 289885.5312
